In [20]:
#!pip install langchain==0.0.309 --quiet

#!pip install langchain==0.0.316  --quiet
#!pip install openai==0.28.1 --quiet

!pip install langchain==0.0.343  --quiet
!pip install Openai==1.3.6  --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.8 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.
spyder-kernels 2.3.3 requires ipython<8,>=7.31.1; python_version >= "3", but you h

In [21]:
#import openai
import langchain
#from langchain import OpenAI
api_key =""
#openai.api_key = api_key

In [22]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message("Can you give me the solution for 10+10")
chat_history.add_ai_message("The Solution is 10+10=20")
chat_history.add_user_message("Thank You")

print(chat_history.messages)


[HumanMessage(content='Can you give me the solution for 10+10'), AIMessage(content='The Solution is 10+10=20'), HumanMessage(content='Thank You')]


# Building a ChatBot with LangChain
## Start with a chat llm object

In [24]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(openai_api_key=api_key)
#chat = OpenAI(openai_api_key=api_key)

chat([HumanMessage(content="Translate this sentence from English to French: I love programming.")])
#chat(prompt="Translate this sentence from English to French: I love programming.")

AttributeError: module 'openai' has no attribute 'error'

Note that the llm object has no memory. It is not "context aware":

In [ ]:
chat([HumanMessage(content="Now in Spanish")])

## Next, use a ConversationChain to add memory/context
...and turn it into a ChatBot

In [ ]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=chat)
conversation.run("Translate this sentence from English to French: I love programming.")

In [ ]:
conversation.run("Now in Sparnish.")

# Working with Memory (Chat Context)

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi!'), AIMessage(content='whats up?')]))

## Let's create a scripted conversation

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")
memory.chat_memory.add_user_message("Answer only in Croatian")
memory.chat_memory.add_ai_message("Understood. I will answer only in Croatian.")
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi!'), AIMessage(content='whats up?'), HumanMessage(content='Answer only in Croatian'), AIMessage(content='Understood. I will answer only in Croatian.')]))

## And then load it into a new ChatBot

In [8]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=chat,memory=memory)
conversation.run("What is an LLM?")

AttributeError: module 'openai' has no attribute 'error'

# Let's look at another approach
## Using a LLMChain, and passing in history through the prompt arg

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain #,ConversationChain

# LLM
llm = ChatOpenAI(openai_api_key=api_key)

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human. Answer only in Croatian."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human. Answer only in Croatian.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Bok! Kako mogu pomoći?')],
 'text': 'Bok! Kako mogu pomoći?'}